In [1]:
import pandas as pd #the csv reading library
import matplotlib.pyplot as plt #the ploting library
import glob #to get file names
import scipy.optimize
from scipy.optimize import curve_fit
from scipy import interpolate
from scipy.interpolate import pchip_interpolate
import numpy as np

In [2]:
# This returns a list of names (strings) of files that have the .txt extension. 
# You should provide the path
files = glob.glob('../../../CDOM_DATA/BLU22/BC_ABS_BLU22_2226_GB/*.txt')
files

['../../../CDOM_DATA/BLU22/BC_ABS_BLU22_2226_GB\\222601_Absorbance__0__00.txt',
 '../../../CDOM_DATA/BLU22/BC_ABS_BLU22_2226_GB\\222601_Absorbance__1__01.txt',
 '../../../CDOM_DATA/BLU22/BC_ABS_BLU22_2226_GB\\222602_Absorbance__0__02.txt',
 '../../../CDOM_DATA/BLU22/BC_ABS_BLU22_2226_GB\\222603_Absorbance__0__03.txt',
 '../../../CDOM_DATA/BLU22/BC_ABS_BLU22_2226_GB\\222604_Absorbance__0__04.txt',
 '../../../CDOM_DATA/BLU22/BC_ABS_BLU22_2226_GB\\222605_Absorbance__0__05.txt',
 '../../../CDOM_DATA/BLU22/BC_ABS_BLU22_2226_GB\\222606_Absorbance__0__06.txt',
 '../../../CDOM_DATA/BLU22/BC_ABS_BLU22_2226_GB\\222607_Absorbance__0__07.txt',
 '../../../CDOM_DATA/BLU22/BC_ABS_BLU22_2226_GB\\222607_Absorbance__0__08.txt',
 '../../../CDOM_DATA/BLU22/BC_ABS_BLU22_2226_GB\\222607_Absorbance__1__09.txt',
 '../../../CDOM_DATA/BLU22/BC_ABS_BLU22_2226_GB\\222607_Absorbance__2__10.txt',
 '../../../CDOM_DATA/BLU22/BC_ABS_BLU22_2226_GB\\222608_Absorbance__0__11.txt',
 '../../../CDOM_DATA/BLU22/BC_ABS_BLU22_

In [3]:
#first Identify what wavegide was used then uncomment the correct waveguide 
X = .488 # 50 cm lwcc number 3 SN: 

In [4]:
#SAMPLE OPENS FILES 
def open_file(name):
    # returns a datafame from a raw file
    # input: filename (string)
    # output: pandas dataframe 
    df = pd.read_csv(name, skiprows=14, sep='\t', names=['wl', 'abs'], )
    return df

In [5]:
#SAMPLE ABS TRANSFORM AND BASLINE CORRECTION 
def abs_base_cor(name):
    # unit conversion
    df['abs'] = df['abs']*(2.303/X)
    # baseline correction
    bcorr = df[(df['wl']>=700) & (df['wl']<=750)]['abs'].mean() # a float
    df['abs'] = df['abs'] - bcorr
    return df

In [6]:
#SAMPLE PCHIPS DATA INTO WHOLE WAVELENTHS 
def Pchip(name):
    #PCHIP 1-D monotonic cubic interpolation.
    #x and y are arrays of values used to approximate some function f, with y = f(x).
    #The interpolant uses monotonic cubic splines to find the value of new points. 
    #PCHIP stands for Piecewise Cubic Hermite Interpolating Polynomial
    x = df['wl']
    y = df['abs']
    #dose the interplation 
    pchip_obj1 = scipy.interpolate.PchipInterpolator(x, y, axis=0, extrapolate=None)
    #pulls the wavelenths data point from the interplation 
    xi = np.arange(189,881,1)
    #pulls the abs values from the wavelenths from the interplation
    yi = pchip_obj1(xi)
    #this turns it back into a dataframe 
    d = {'wl': xi, 'abs': yi}
    df = pd.DataFrame(d)
    return df

In [7]:
#FINAL FUNTION 
#see above for the breakdown of diff parts 
def GET_CDOM_DATA(name):
    df = pd.read_csv(name, skiprows=14, sep='\t', names=['wl', 'abs'], )
    df['abs'] = df['abs']*(2.303/X)
    bcorr = df[(df['wl']>=700) & (df['wl']<=750)]['abs'].mean() # a float
    df['abs'] = df['abs'] - bcorr
    x = df['wl']
    y = df['abs'] 
    pchip_obj1 = scipy.interpolate.PchipInterpolator(x, y, axis=0, extrapolate=None)
    xi = np.arange(189,881,1)
    yi = pchip_obj1(xi)
    d = {'wl': xi, 'abs': yi}
    df = pd.DataFrame(d)
    return df

In [8]:
#TEST FUNCTION 
GET_CDOM_DATA('../../../CDOM_DATA/BLU22/BC_ABS_BLU22_2226_GB/222601_Absorbance__0__00.txt')

,wl,abs
0,189,0.022805
1,190,0.022805
2,191,0.022805
3,192,-1.443088
4,193,-3.036069
...,...,...
687,876,0.239421
688,877,0.334392
689,878,0.369251
690,879,0.679775


In [10]:
#MAKE IT ALL IN ONE EASY TO USE DATA FRAME 
#this makes a new data frame there all the columns are the scanns and the rows are the wavelths 
frames = []
#fig, ax = plt.subplots(figsize=(10,8))
for f in files:
    df = GET_CDOM_DATA(f) # runs the processing function 
    ID = f.split("/")[5][21:-4] #takes junk out of the file name to make a single ID 
    df = df.rename(columns={'abs': ID}) # renames the abs col to the new id 
    df = df.set_index('wl', drop=True) # makes the wl the new index, aka the rows 
    #df.plot(label='', ax=ax)
    frames.append(df) # adds the new col from each scan to a part of frames 
    
df = pd.concat(frames, axis=1, ignore_index=False) # adds all the things in the frame together to make a new df 
df

#ax.set_ylim(-.1,2)
#ax.set_xlim(260,700)
#ax.legend().remove()

,222601_Absorbance__0__00,222601_Absorbance__1__01,222602_Absorbance__0__02,222603_Absorbance__0__03,222604_Absorbance__0__04,222605_Absorbance__0__05,222606_Absorbance__0__06,222607_Absorbance__0__07,222607_Absorbance__0__08,222607_Absorbance__1__09,...,222616_Absorbance__2__40,222616_Absorbance__3__41,222616_Absorbance__4__42,222616_Absorbance__5__43,222617_Absorbance__0__44,222617_Absorbance__1__45,222617_Absorbance__2__46,222618_Absorbance__0__47,222618_Absorbance__1__48,222618_Absorbance__2__49
wl,,,,,,,,,,,,,,,,,,,,,
189,0.022805,-0.547479,-0.002470,-0.060269,-4.175741,-5.749830,-6.665337,-0.070496,0.053776,0.052984,...,0.012470,8.279599,2.659657,3.089346,0.630389,0.026392,-3.725622,-2.979476,-2.100968,-3.202479
190,0.022805,-0.547479,-0.002470,-0.060269,-4.175741,-5.749830,-6.665337,-0.070496,0.053776,0.052984,...,0.012470,8.279599,2.659657,3.089346,0.630389,0.026392,-3.725622,-2.979476,-2.100968,-3.202479
191,0.022805,-1.508002,-0.002470,-0.060269,-0.101602,-2.649157,-3.248219,-0.400090,0.037991,0.052984,...,0.012470,1.281695,3.419984,1.735670,-5.209094,-5.769292,-5.456093,-7.061530,-4.608873,-6.723907
192,-1.443088,-1.891977,0.647073,-5.665583,-0.101602,-0.023005,-0.010233,-0.070496,0.053776,0.052984,...,-0.210898,2.766404,1.444443,0.002138,0.115878,-2.697363,1.099842,-0.001150,0.000048,-0.001876
193,-3.036069,-0.952122,0.383387,-0.060269,-0.522955,-0.625816,-0.747786,0.299787,-1.232864,-1.441592,...,0.445790,1.863742,1.414835,2.718205,0.308459,1.127578,-0.589611,0.191757,-1.312170,-0.261266
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
876,0.239421,0.434345,0.162717,1.303675,0.286829,1.023134,1.362352,-0.053346,0.263967,0.060022,...,0.434556,0.613298,0.507898,0.592012,0.140486,-0.079359,-0.021750,0.176133,0.085911,0.250841
877,0.334392,0.004067,0.418175,1.019832,0.445876,0.733688,1.693085,0.106538,0.516840,0.141460,...,0.427530,0.488813,0.354731,0.492610,-0.134982,-0.014945,0.052678,0.060838,-0.039194,0.066829
878,0.369251,-0.050535,0.309444,1.076627,0.217449,1.323096,1.950167,0.291172,0.228691,0.220303,...,0.289902,0.650817,0.350477,0.397318,0.019251,-0.036957,-0.127372,0.421118,0.100435,0.102201


In [40]:
#HERE WE ARE GOING TO AVERAGE THE SCANNS TO GET A FINAL VALUE FOR EACH WAVELENTH
#this filters eveything by all that have the work 'Absorbance' with anything after it 
#and turns it into a new object 
d = df.filter(regex='Absorbance*', axis=1)
#this splits the column names by 
names = d.columns.str.rsplit('_', n=5).str[0]
#this part groups evethign by the sample ID that we have in names then avarages all scanns regardless of how many there are 
df = d.groupby(names, axis=1).mean()
df

,222601,222602,222603,222604,222605,222606,222607,222608,222609,222610,222611,222612,222613,222614,222615,222616,222617,222618
wl,,,,,,,,,,,,,,,,,,
189,-0.262337,-0.002470,-0.060269,-4.175741,-5.749830,-6.665337,0.021758,0.041221,-0.007434,-0.573423,-0.041142,-0.067953,-0.375929,-0.044833,0.060942,2.345574,-1.022947,-2.760975
190,-0.262337,-0.002470,-0.060269,-4.175741,-5.749830,-6.665337,0.021758,0.041221,-0.007434,-0.573423,-0.041142,-0.067953,-0.375929,-0.044833,0.060942,2.345574,-1.022947,-2.760975
191,-0.742599,-0.002470,-0.060269,-0.101602,-2.649157,-3.248219,-0.064587,0.041221,-0.007434,-2.645294,-1.784385,-4.239753,-1.272274,-0.396562,0.076491,1.080365,-5.478160,-6.131437
192,-1.667533,0.647073,-5.665583,-0.101602,-0.023005,-0.010233,0.021758,0.041221,-0.007434,-0.058394,1.402589,-0.067953,0.663148,0.120472,-0.299441,0.601662,-0.493881,-0.000993
193,-1.994095,0.383387,-0.060269,-0.522955,-0.625816,-0.747786,-0.972829,-2.625235,-3.429944,2.614364,0.527039,0.898282,-0.915547,0.007295,0.142124,1.171280,0.282142,-0.460560
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
876,0.336883,0.162717,1.303675,0.286829,1.023134,1.362352,0.138756,0.534828,0.949026,0.323598,0.284904,0.288668,0.283858,0.164649,0.302607,0.463856,0.013126,0.170962
877,0.169229,0.418175,1.019832,0.445876,0.733688,1.693085,0.247260,0.609861,0.943834,0.213698,0.287808,0.457723,0.148855,0.230638,0.374307,0.462982,-0.032416,0.029491
878,0.159358,0.309444,1.076627,0.217449,1.323096,1.950167,0.273711,0.651043,0.921366,0.298651,0.446118,0.716867,0.125138,0.256138,0.264939,0.429467,-0.048359,0.207918


In [44]:
#here we make a copy of the data that is vertical so Jordon likes it 
Abs_data = df.transpose(copy=False)
Abs_data
#save it as a CSV to the propor location with the correct file name

##Abs_data.to_csv('../outputs/BC_ABS_BLU22_2226_GB_wavelenths.csv')

I need an output table for each sampling event with the sample number
corresponding parameters = abs 254, abs 280, abs 320, abs 412, abs 444, the spectral slope of 275 to 295, the spectral slope of 350 to 400, the spectral slope of 275 to 700, and the baseline correction we used, aka average of 700 to 750 nm. 

In [71]:
#MAKE AN EMPTY DATA FRAME WITH WHAT YOU NEED  
outputs = pd.DataFrame(index=['abs_254','abs_280','abs_320','abs_412','abs_444',
                             'ss_275_295', 'ss_350_400', 'ss_275_700', 'baseline_correction'])
outputs = outputs.reindex(columns=df.columns) #copies the col names from the processed CDOM data 
outputs

,222601,222602,222603,222604,222605,222606,222607,222608,222609,222610,222611,222612,222613,222614,222615,222616,222617,222618
abs_254,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
abs_280,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
abs_320,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
abs_412,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
abs_444,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ss_275_295,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ss_350_400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ss_275_700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
baseline_correction,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [105]:
abs_254 = df[65:66]
abs_280 = df[91:92]
abs_320 = df[131:132]
abs_412 = df[223:224]
abs_444 = df[255:256]

abs_254

,222601,222602,222603,222604,222605,222606,222607,222608,222609,222610,222611,222612,222613,222614,222615,222616,222617,222618
wl,,,,,,,,,,,,,,,,,,
254,8.613115,1.761131,2.03178,7.130295,6.881487,2.574486,1.922783,3.655796,3.710646,2.301177,2.178483,3.123997,2.82853,2.165088,2.131942,2.978602,2.692227,2.184315


In [87]:
outputs['abs_254'] = abs_254

ValueError: Expected a 1D array, got an array with shape (9, 18)

In [ ]:
#we too three scans of the samples, and we want to average the scanns 
#we will use the underscore in the file name to separete the averages 
# set the nummber to the approprate number of undercores in the same to seperate by the sample ID or Standterd 

sample = np.unique([f.split('_')[5] for f in files])
sample